In [64]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
import pandas as pd

import sklearn.model_selection as sc_model_selection


In [66]:
data = pd.read_csv("data/train.csv")

In [67]:
data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


# Preprocessing

In [68]:
from pipeline.pipeline import impute, make_pipeline
data = impute(data)
data = make_pipeline(data)

/workspaces/kaggle-actions/spaceship_titanic/spaceship/pipeline/pipeline.py:45: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.


# Training

In [69]:
X = data.loc[:, data.columns != 'Transported']
Y = data['Transported']
X_tmp, X_test = sc_model_selection.train_test_split(X)
Y_tmp, Y_test = sc_model_selection.train_test_split(Y)
X_train, X_val = sc_model_selection.train_test_split(X_tmp)
Y_train, Y_val = sc_model_selection.train_test_split(Y_tmp)


# Definitions

In [70]:
data.columns.values[0:1000]

array(['PassengerId', 'Cabin', 'Age', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported',
       'No_Cabin_Indicator', 'No_Destination_Indicator',
       'RoomService_paid', 'FoodCourt_paid', 'HomePlanet_Earth',
       'HomePlanet_Europa', 'HomePlanet_Mars', 'VIP_False', 'VIP_True',
       'Destination_55 Cancri e', 'Destination_PSO J318.5-22',
       'Destination_TRAPPIST-1e', 'Num', 'Deck_A', 'Deck_B', 'Deck_C',
       'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'Deck_T', 'Side_P',
       'Side_S', 'CryoSleep_False', 'CryoSleep_True'], dtype=object)

In [71]:
COLUMNS = [
    ## Metric
    "Age",
    "RoomService",
    "RoomService_paid",
    "FoodCourt",
    "FoodCourt_paid",
    #"ShoppingMall",
    "Spa",
    "VRDeck",
    "Num",
    ## HomePlanet
    #"HomePlanet_Earth",
    "HomePlanet_Europa",
    #"HomePlanet_Mars",
    ## VIP
    "VIP_True",
    "VIP_False",
    ## Dest
    "Destination_55 Cancri e",
    #"Destination_PSO J318.5-22",
    #"Destination_TRAPPIST-1e",
    ## Cryo
    "CryoSleep_False",
    #"CryoSleep_True",
    ## Deck
    #"Deck_A",
    "Deck_B",
    "Deck_C",
    #"Deck_D",
    "Deck_E",
    "Deck_F",
    "Deck_G",
    "Deck_T",
    ## Side
    "Side_P",
    "Side_S",
    "No_Cabin_Indicator"
]


## Models

In [72]:
from models.baseline_regression import get_baseline_regression_model
from models.baseline_nn import get_baseline_nn_model
model = get_baseline_regression_model(X_train, Y_train, COLUMNS)
#model = get_baseline_nn_model(X_train, Y_train, COLUMNS)

# Evaluation on validation data

In [73]:
from sklearn.metrics import accuracy_score
prediction = model.predict(X_val[COLUMNS])
accuracy_score(prediction, Y_val)

0.4723926380368098

# Train on full model

In [74]:
model = get_baseline_nn_model(X, Y, COLUMNS)

# Prediction

In [75]:
data_predict = pd.read_csv("data/test.csv")
data_predict = impute(data_predict)
data_predict = make_pipeline(data_predict)
# Get features
predictor = data_predict[COLUMNS]
# Predict
PassengerId = data_predict["PassengerId"]
Prediction = model.predict(predictor)

/workspaces/kaggle-actions/spaceship_titanic/spaceship/pipeline/pipeline.py:45: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.


# Write to file

In [76]:
pd.DataFrame(Prediction, index=PassengerId, columns=["Transported"]).to_csv("my_submission.csv")